In [200]:
# imports
import sys
import pandas as pd
from Bio.KEGG.REST import *
from Bio.KEGG.KGML import KGML_parser
#from Bio.Graphics.KGML_vis import KGMLCanvas
#from Bio import Entrez

from IPython.display import Image

from io import StringIO

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import scipy.stats as stats
from statsmodels.sandbox.stats.multicomp import multipletests

## KEGG REST service and visualizing KEGG pathways
You can find documentation about the REST service itself and how to use it directly using URLs in a browser here: http://www.genome.jp/kegg/rest/keggapi.html

The following link shows a Jupyter notebook that contains code examples that are very similar to the tasks in this Pathways notebook, and almost in the same order, so take a look if you need help with anything from access to the KEGG REST service using Biopython to coloring nodes in a pathway visualization:
http://nbviewer.jupyter.org/github/widdowquinn/notebooks/blob/master/Biopython_KGML_intro.ipynb

## Task 1: KEGG and gene id mapping

Familiarize yourself with the KEGG Rest interface and how to access it with Biopyhton.<br>
Before you start solving task 1.1, read its task description, look at and run all code examples given there. Try to understand what happens, maybe by recreating some REST calls as URL calls in your browser.<br>
The Biopython functions `kegg_list` and `kegg_link` simply wrap this URL call into a function and return the response in plain text.<br>
E.g. accessing the URL http://rest.kegg.jp/list/pathway/hsa lists all pathways associated with humans and
is equivalent to calling<br>
`kegg_list('pathway', 'hsa').read()` in Python.<br>
(Organsim IDs: hsa = Homo Sapiens, mmu = Mus musculus (mouse))

In [201]:
print(kegg_list('pathway', 'hsa').read())

path:hsa00010	Glycolysis / Gluconeogenesis - Homo sapiens (human)
path:hsa00020	Citrate cycle (TCA cycle) - Homo sapiens (human)
path:hsa00030	Pentose phosphate pathway - Homo sapiens (human)
path:hsa00040	Pentose and glucuronate interconversions - Homo sapiens (human)
path:hsa00051	Fructose and mannose metabolism - Homo sapiens (human)
path:hsa00052	Galactose metabolism - Homo sapiens (human)
path:hsa00053	Ascorbate and aldarate metabolism - Homo sapiens (human)
path:hsa00061	Fatty acid biosynthesis - Homo sapiens (human)
path:hsa00062	Fatty acid elongation - Homo sapiens (human)
path:hsa00071	Fatty acid degradation - Homo sapiens (human)
path:hsa00072	Synthesis and degradation of ketone bodies - Homo sapiens (human)
path:hsa00100	Steroid biosynthesis - Homo sapiens (human)
path:hsa00120	Primary bile acid biosynthesis - Homo sapiens (human)
path:hsa00130	Ubiquinone and other terpenoid-quinone biosynthesis - Homo sapiens (human)
path:hsa00140	Steroid hormone biosynthesis - Homo sapiens

### Subtask 1.1 Extract gene lists for all (mouse) KEGG pathways and store them
Below is some example code showing how to get data out of the KEGG REST service in general.
It lists all the mouse pathways and extracts the pathway IDs from the REST response, then pulls the gene list for one pathway.

* The raw list of pathways looks like this:<br>
path:mmu00010	Glycolysis / Gluconeogenesis - Mus musculus (mouse)<br>
path:mmu00020	Citrate cycle (TCA cycle) - Mus musculus (mouse)<br>
path:mmu00030	Pentose phosphate pathway - Mus musculus (mouse)<br>
path:mmu00040	Pentose and glucuronate interconversions - Mus musculus (mouse)<br>
path:mmu00051	Fructose and mannose metabolism - Mus musculus (mouse)<br>
path:mmu00052	Galactose metabolism - Mus musculus (mouse)<br>
path:mmu00053	Ascorbate and aldarate metabolism - Mus musculus (mouse)<br>
...<br>
(path:ID)(TAB)(Description with spaces)


* Each raw list of genes looks like this:<br>
path:mmu00010	mmu:100042025<br>
path:mmu00010	mmu:103988<br>
path:mmu00010	mmu:106557<br>
path:mmu00010	mmu:110695<br>
path:mmu00010	mmu:11522<br>
path:mmu00010	mmu:11529<br>
path:mmu00010	mmu:11532<br>
...<br>
(path:ID)(TAB)(organismID:geneID)
<br>

The gene identifiers are so-called Entrez ID's: KEGG chose to use the Entrez system.

* Extend the code to get lists of gene IDs for each pathway<br>
* Store the lists in a way that allows convenient lookup of all genes in each pathway:
A DataFrame with two columns: Pathway and Entrez, where the pathway ID will be equal for all genes from one pathway. From this you can extract all genes of a pathway using the first column. Be aware that none of those columns on their own can be a unique identifier for each row. Each gene might also be present in multiple pathways: it's a typical case of N-to-N ("many-to-many") mapping.
 
BEWARE! You will need to do many calls to the REST service to retrieve pathway information and this might take a lot of time, partly because of restrictions to the number of calls per second allowed by Biopython.<br>
Make sure your code works as expected on e.g. 3 Pathways before you start to process all (> 300) of them.

In [202]:
# get all pathways from organism 'mmu' (Mus musculus)
pathwaysResponse = kegg_list('pathway', 'mmu').read()
# Format = path:ID Description
print(pathwaysResponse)
pathwaysResponse

path:mmu00010	Glycolysis / Gluconeogenesis - Mus musculus (mouse)
path:mmu00020	Citrate cycle (TCA cycle) - Mus musculus (mouse)
path:mmu00030	Pentose phosphate pathway - Mus musculus (mouse)
path:mmu00040	Pentose and glucuronate interconversions - Mus musculus (mouse)
path:mmu00051	Fructose and mannose metabolism - Mus musculus (mouse)
path:mmu00052	Galactose metabolism - Mus musculus (mouse)
path:mmu00053	Ascorbate and aldarate metabolism - Mus musculus (mouse)
path:mmu00061	Fatty acid biosynthesis - Mus musculus (mouse)
path:mmu00062	Fatty acid elongation - Mus musculus (mouse)
path:mmu00071	Fatty acid degradation - Mus musculus (mouse)
path:mmu00072	Synthesis and degradation of ketone bodies - Mus musculus (mouse)
path:mmu00100	Steroid biosynthesis - Mus musculus (mouse)
path:mmu00120	Primary bile acid biosynthesis - Mus musculus (mouse)
path:mmu00130	Ubiquinone and other terpenoid-quinone biosynthesis - Mus musculus (mouse)
path:mmu00140	Steroid hormone biosynthesis - Mus musculus

'path:mmu00010\tGlycolysis / Gluconeogenesis - Mus musculus (mouse)\npath:mmu00020\tCitrate cycle (TCA cycle) - Mus musculus (mouse)\npath:mmu00030\tPentose phosphate pathway - Mus musculus (mouse)\npath:mmu00040\tPentose and glucuronate interconversions - Mus musculus (mouse)\npath:mmu00051\tFructose and mannose metabolism - Mus musculus (mouse)\npath:mmu00052\tGalactose metabolism - Mus musculus (mouse)\npath:mmu00053\tAscorbate and aldarate metabolism - Mus musculus (mouse)\npath:mmu00061\tFatty acid biosynthesis - Mus musculus (mouse)\npath:mmu00062\tFatty acid elongation - Mus musculus (mouse)\npath:mmu00071\tFatty acid degradation - Mus musculus (mouse)\npath:mmu00072\tSynthesis and degradation of ketone bodies - Mus musculus (mouse)\npath:mmu00100\tSteroid biosynthesis - Mus musculus (mouse)\npath:mmu00120\tPrimary bile acid biosynthesis - Mus musculus (mouse)\npath:mmu00130\tUbiquinone and other terpenoid-quinone biosynthesis - Mus musculus (mouse)\npath:mmu00140\tSteroid hormo

In [203]:
# split the response on "\n" newline characters, split the lines on "\t" characters
# store them in a suitable data structure (e.g. a Series indexed by pathway ID's)
def parse(in_str):
    to_parse = StringIO(in_str)
    return pd.read_csv(to_parse, sep = '\t', header = None)
    
pathway_resp_df = parse(pathwaysResponse)

display(pathway_resp_df)

# pw_ids_names = ...

,0,1
0,path:mmu00010,Glycolysis / Gluconeogenesis - Mus musculus (m...
1,path:mmu00020,Citrate cycle (TCA cycle) - Mus musculus (mouse)
2,path:mmu00030,Pentose phosphate pathway - Mus musculus (mouse)
3,path:mmu00040,Pentose and glucuronate interconversions - Mus...
4,path:mmu00051,Fructose and mannose metabolism - Mus musculus...
5,path:mmu00052,Galactose metabolism - Mus musculus (mouse)
6,path:mmu00053,Ascorbate and aldarate metabolism - Mus muscul...
7,path:mmu00061,Fatty acid biosynthesis - Mus musculus (mouse)
8,path:mmu00062,Fatty acid elongation - Mus musculus (mouse)
9,path:mmu00071,Fatty acid degradation - Mus musculus (mouse)


In [204]:
# use kegg_link(organism, pathway).read() to get the list of genes for each pathway

# example for one pathway:
exampleGeneList = kegg_link('mmu', 'path:mmu00010').read()
# print(exampleGeneList)

# A neat trick to turn this (and the list of pathways before) directly into a pandas DataFrame
# is to use StringIO to simulate a CSV input file:
tmp_df = pd.read_csv(StringIO(exampleGeneList), sep='\t', header=None)
tmp_df

path = pathway_resp_df.iloc[0,0]
path_str = kegg_link('mmu', path).read()
all_df = pd.read_csv(StringIO(path_str), sep='\t', header=None)
for path in pathway_resp_df.iloc[1:,0]:
    path_str = kegg_link('mmu', path).read()
    path_df = pd.read_csv(StringIO(path_str), sep='\t', header=None)
    all_df = pd.concat([all_df, path_df])

all_df
# Extend this code so that you obtain all pathway tables, and concatenate them into a single
# pathway_entrez DataFrame.

,0,1
0,path:mmu00010,mmu:100042025
1,path:mmu00010,mmu:103988
2,path:mmu00010,mmu:106557
3,path:mmu00010,mmu:110695
4,path:mmu00010,mmu:11522
5,path:mmu00010,mmu:11529
6,path:mmu00010,mmu:11532
7,path:mmu00010,mmu:11669
8,path:mmu00010,mmu:11670
9,path:mmu00010,mmu:11671


In [205]:
all_df = all_df.rename(columns = {0:'pathway', 1:'Entrez'})

In [206]:
backup_all = all_df.copy()
#all_df = backup_all.copy()

all_df.iloc[:,1] = all_df.iloc[:,1].str.replace('mmu:', '')
all_df.iloc[:,1] = all_df.iloc[:,1].apply(int)
all_df.head()

,pathway,Entrez
0,path:mmu00010,100042025
1,path:mmu00010,103988
2,path:mmu00010,106557
3,path:mmu00010,110695
4,path:mmu00010,11522


In [207]:
#kegg_link('mmu', 'path:mmu00010').read()

In [208]:
#all_df['Entrez']

### Subtask 1.2: Create the pathway-to-gene DataFrame
How many rows does the `pathway_entrez` DataFrame have? How many unique Entrez identifiers are in there?
Store the DataFrame as a csv file so that you can load it back up easily if necessary.


In [209]:
print('Shape Dataframe', np.shape(all_df))
entrez_arr = all_df['Entrez'].values
display(all_df.nunique())
#unique, counts = np.unique(entrez_arr, return_counts=True)
#cts = dict(zip(unique, counts))


Shape Dataframe (31697, 2)


pathway     326
Entrez     8598
dtype: int64

The dataframe has 31697 Entries.
There are 8598 unique Entrez identifiers.

### Subtask 1.3: Create a conversion table between gene identifier formats

http://www.informatics.jax.org/downloads/reports/MGI_Gene_Model_Coord.rpt <br>
The file above contains mappings between different identifier types, including the Entrez IDs that KEGG uses and the gene symbols we have in the DE data.
Download and read in this file, then use the respective columns to map the Entrez ID numbers to gene symbols.

Create a DataFrame with two columns:
* `Gene.1` (to match the index name we had used in the DE notebook) and
* `Entrez` to match our KEGG `pathway_entrez` table.

Watch out! Pandas may convert the Entrez ID's to floating point numbers when loading the csv. Also, the "mmu:" prefix which KEGG uses is missing from them. Turn those `12345.0`-like floating point values to `mmu:12345` strings, otherwise you will have a hard time matching them with KEGG pathways.

In [210]:
rpt_df = pd.read_csv('Day2_Files\\MGI_Gene_Model_Coord.txt', sep = '\t', index_col = False)
rpt_df.head()

,1. MGI accession id,2. marker type,3. marker symbol,4. marker name,5. genome build,6. Entrez gene id,7. NCBI gene chromosome,8. NCBI gene start,9. NCBI gene end,10. NCBI gene strand,11. Ensembl gene id,12. Ensembl gene chromosome,13. Ensembl gene start,14. Ensembl gene end,15. Ensembl gene strand
0,MGI:87853,Gene,a,nonagouti,GRCm38,50518.0,2,154950204.0,155051012.0,+,ENSMUSG00000027596,2,154791402.0,155051012.0,+
1,MGI:87854,Gene,Pzp,"PZP, alpha-2-macroglobulin like",GRCm38,11287.0,6,128483567.0,128526720.0,-,ENSMUSG00000030359,6,128483567.0,128526720.0,-
2,MGI:87859,Gene,Abl1,"c-abl oncogene 1, non-receptor tyrosine kinase",GRCm38,11350.0,2,31688354.0,31807093.0,+,ENSMUSG00000026842,2,31688376.0,31804227.0,+
3,MGI:87860,Gene,Abl2,v-abl Abelson murine leukemia viral oncogene 2...,GRCm38,11352.0,1,156558171.0,156649619.0,+,ENSMUSG00000026596,1,156558786.0,156649568.0,+
4,MGI:87862,Gene,Scgb1b27,"secretoglobin, family 1B, member 27",GRCm38,11354.0,7,34021567.0,34022881.0,+,ENSMUSG00000066583,7,34021483.0,34022881.0,+


In [211]:
#rpt_df.columns.tolist()
conv_table = rpt_df[['3. marker symbol', '6. Entrez gene id']]
conv_table = conv_table.dropna(axis = 0)
conv_table = conv_table.rename(columns = {'3. marker symbol': 'Gene.1', '6. Entrez gene id': 'Entrez'})
#conv_table[]
conv_table.head()

,Gene.1,Entrez
0,a,50518.0
1,Pzp,11287.0
2,Abl1,11350.0
3,Abl2,11352.0
4,Scgb1b27,11354.0


In [212]:
conv_table['Entrez'] = conv_table['Entrez'].apply(int)
conv_table.head()
#conv_table_fin = conv_table.copy()
#for i in range(np.shape(conv_table)[0]):
#    conv_table_fin.iloc[i,0] = str(conv_table_fin.iloc[i,1]) + str(conv_table_fin.iloc[i,0])
#conv_table_fin.head()

,Gene.1,Entrez
0,a,50518
1,Pzp,11287
2,Abl1,11350
3,Abl2,11352
4,Scgb1b27,11354


## Task 2: Gene Set Enrichment

### Subtask 2.1: Prepare your differential expression data

* Read in the csv file you produced in the Differential Expression notebook
* Introduce a new boolean column `significant`, with criteria that you can adjust with later. You can start with -log10p > 2 and |log2fold| > 0.5 but they are not necessarily your final values.

In [213]:
p_vals = pd.read_csv('Day2_Files//p_val_dat.csv')
#Merging stuff:
merged = p_vals.merge(conv_table, on = 'Gene.1')
merged.head()
merged = merged.merge(all_df, on = 'Entrez')

merged_copy = merged.copy()
merged['significant'] = (merged['p_val_corr_log'] > 2) & (np.abs(merged['log2fc']) > 0.5)
display(merged.head())

,Gene.1,log2fc,p_val,p_corr,p_val_log,p_val_corr_log,Entrez,pathway,significant
0,1300017J02Rik,0.101275,0.001076,0.003552,2.968086,2.449528,71775,path:mmu04066,False
1,1300017J02Rik,0.101275,0.001076,0.003552,2.968086,2.449528,71775,path:mmu04216,False
2,1300017J02Rik,0.101275,0.001076,0.003552,2.968086,2.449528,71775,path:mmu04978,False
3,1700009N14Rik,0.010075,0.797881,0.853212,0.098062,0.068943,75471,path:mmu03008,False
4,1700009N14Rik,0.010075,0.797881,0.853212,0.098062,0.068943,75471,path:mmu03013,False


### Subtask 2.2: Perform gene set enrichment with the KEGG gene sets you extracted in Task 1
Several tests are suitable for our purpose: the $\chi^2$ test we had used before, Fisher's exact test, or the simplest binomial test.<br>

Pick one of them. If you choose a contingency-table based test ($\chi^2$ or Fisher's exact), find a way to calculate the entries of the contingency table for each pathway (significantly DE and contained in PW / not significant and contained / significant and not contained / not significant and not contained).

If you choose a binomial test, determine the successes / number of trials / probability of success parameters. And remember, the latter is independent of your pathway! This is why it's a simplification.

In [214]:
pathways = merged['pathway'].tolist()
unique, counts = np.unique(pathways, return_counts=True)
cts = dict(zip(unique, counts))
#merged = merged.index(index = 'pathway', append = True)
#merged.set_index['pathway']
#display(cts.keys())
merged_nodub = merged.drop_duplicates('Gene.1')
prob_success = np.shape(merged_nodub[merged_nodub['significant'] == True])[0] / np.shape(merged_nodub)[0]
display(prob_success)
binom_out = {}
for key in cts.keys():
    this_path = merged.loc[merged['pathway'] == key]
    successes = np.shape(this_path[this_path['pathway'] == True])[0]
    num_trials = np.shape(this_path)[0]
    test_res = stats.binom_test(successes, n = num_trials, p = prob_success)
    binom_out[key] = test_res
    
    #stats.


0.006602403274792024

In [215]:
test_res = pd.DataFrame.from_dict(binom_out, orient = 'index')

test_res = test_res.rename(columns={0:'p_val_binom'})
test_res.head()

,p_val_binom
path:mmu00010,1.0
path:mmu00020,1.0
path:mmu00030,1.0
path:mmu00040,1.0
path:mmu00051,1.0


### Subtask 2.3: Extract a list of significantly enriched KEGG pathways
How many pathways are enriched? How many survive the Benjamini-Hochberg correction? Do you have to update your criteria for gene significance?

Is the proportion of enriched pathways similar to the proportion of differentially expressed genes?

In [216]:
test_res_sig = test_res[test_res['p_val_binom']< 0.05]

test_res_sig

,p_val_binom
path:mmu01100,0.000876
path:mmu04740,0.001760


There are two pathways with a significantly low p-value. Those genes are assumed to be differentially expressed

In [218]:
def benj_test(p_val_chr_df):
    p_val_chr_df['p_corr_binom'] = multipletests(p_val_chr_df['p_val_binom'], method ='fdr_bh', alpha = 0.05)[1]
    
benj_test(test_res)
#display(test_res)
test_res_sig_corr = test_res[test_res['p_corr_binom']< 0.05]
#test_res_sig_corr

### Subtask 2.4: Create a pathway DataFrame combining all of the above
* indexed by pathway ID
* number of genes in pathway
* number of significant genes in pathway
* p-values of enrichment

In [232]:

display(test_res.head())

,p_val_binom,p_corr_binom
path:mmu00010,1.0,1.0
path:mmu00020,1.0,1.0
path:mmu00030,1.0,1.0
path:mmu00040,1.0,1.0
path:mmu00051,1.0,1.0


In [240]:
#index = []
num_genes = []
num_sig_genes = []

all_df = test_res.copy()

for en in test_res.index.tolist():
    
    this_path = merged.loc[merged['pathway'] == en]
    num_genes.append(np.shape(this_path)[0])
    num_sig_genes.append( sum(this_path['significant']))
all_df['num_genes'] = num_genes
all_df['num_sig_genes'] = num_sig_genes
display(all_df.head())

all_df.to_csv('pathway_data\\pathway_all_df.csv')

    
    
    
#this_path
#merged['pathway']  test_res_sig.index.tolist()

,p_val_binom,p_corr_binom,num_genes,num_sig_genes
path:mmu00010,1.000000,1.0,61,0
path:mmu00020,1.000000,1.0,31,0
path:mmu00030,1.000000,1.0,30,0
path:mmu00040,1.000000,1.0,24,0
path:mmu00051,1.000000,1.0,33,0
path:mmu00052,1.000000,1.0,30,0
path:mmu00053,1.000000,1.0,19,0
path:mmu00061,1.000000,1.0,14,0
path:mmu00062,1.000000,1.0,25,1
path:mmu00071,1.000000,1.0,45,0


In [237]:
display(sum(this_path['significant']))

1